In [1]:
import pandas as pd
import pybaseball as pb
from functools import partial

In [4]:
# statcast only retunrs 40000 results, so we'll need to page through the results
def page_statcast(start_dt, end_dt):
    dates = pd.date_range(start_dt, end_dt, freq="W")
    start_end_pairs = list(zip(dates, dates.shift()))
    df = pd.concat([ pb.statcast(start_dt.strftime("%Y-%m-%d"),
                                end_dt.strftime("%Y-%m-%d"))
                   for start_dt, end_dt in start_end_pairs
                   ])
    try:
        df["pitch_id"] = df.game_pk.astype(str) + df.pitcher.astype(str) + df.pitch_number.astype(str)
        df = df.drop_duplicates(subset=["pitch_id"])
    except:
        pass
    return df

In [5]:
df = page_statcast('1-1-2020', '12-12-2020')

# df = pd.read_pickle("statcast_2020_all.pkl")


In [6]:
# you'll probably want to save this big-ass data pull
# df.to_pickle("statcast_2020_all.pkl")

In [7]:
import numpy as np

In [8]:
def filter_cols(df):
    cols = [
     'game_date',
     'player_name',
     'pitch_type',
     'release_speed',
     'description',
     'pitch_number',
     'zone',
     'release_pos_x',
     'release_pos_z',
     'batter',
     'pitcher',
     'events',
     'des',
     'game_type',
     'stand',
     'p_throws',
     'home_team',
     'away_team',
     'type','plate_x',
     'plate_z', 'sz_top',
     'sz_bot', 'game_pk'
    ]
    return df[cols]

In [9]:
def add_sz_center(df):
    df["sz_center"] = (df.sz_top + df.sz_bot) / 2
    return df

def add_pnts(df):
    df["pitch_plate"] = [ tuple(x) for x in df[["plate_x", "plate_z"]].itertuples(index=False, name=None) ]
    df["center_plate"] = [ tuple(x) for x in np.array([np.zeros(df.shape[0]), df.sz_center]).T ]
    return df

def euc_dist_center(row):
    point1 = np.array(row["pitch_plate"])
    point2 = np.array(row["center_plate"])
    dist = np.linalg.norm(point1 - point2)
    return dist
    

In [10]:
df = df.pipe(filter_cols).pipe(add_sz_center).pipe(add_pnts)

/home/jup/miniconda3/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/home/jup/miniconda3/lib/python3.7/site-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/home/jup/miniconda3/lib/python3.7/site-packages/ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://p

In [11]:
%%time
df["dist_center"] = df.apply(euc_dist_center, axis=1)

CPU times: user 3.75 s, sys: 70.8 ms, total: 3.83 s
Wall time: 3.8 s


In [12]:
df.sort_values("dist_center", ascending=False).head(20)

,game_date,player_name,pitch_type,release_speed,description,pitch_number,zone,release_pos_x,release_pos_z,batter,...,type,plate_x,plate_z,sz_top,sz_bot,game_pk,sz_center,pitch_plate,center_plate,dist_center
5070,2020-09-05,Shun Yamaguchi,CS,49.8,ball,1.0,14.0,-1.17,5.50,622168.0,...,B,15.21,-28.48,3.28,1.61,631625.0,2.445,"(15.21, -28.48)","(0.0, 2.445)",34.463020
25464,2020-09-01,Richard Rodriguez,SL,81.7,ball,1.0,14.0,-3.23,5.80,595879.0,...,B,4.16,-2.85,3.30,1.60,631109.0,2.450,"(4.16, -2.85)","(0.0, 2.45)",6.737626
15178,2020-09-17,Shane Bieber,KC,85.6,ball,5.0,14.0,-1.00,5.61,656537.0,...,B,2.82,-2.59,3.47,1.67,631464.0,2.570,"(2.82, -2.59)","(0.0, 2.5700000000000003)",5.880306
24018,2020-09-22,Ryan Tepera,FC,90.2,ball,1.0,14.0,-2.03,5.69,663647.0,...,B,2.98,-2.53,3.30,1.50,631088.0,2.400,"(2.98, -2.53)","(0.0, 2.4)",5.760668
181,2020-08-23,Zack Godley,FC,86.6,ball,3.0,11.0,-1.18,6.18,663624.0,...,B,-4.23,6.46,3.59,1.76,631670.0,2.675,"(-4.23, 6.46)","(0.0, 2.675)",5.676189
14842,2020-09-24,Carlos Rodon,SL,81.5,ball,1.0,12.0,1.70,6.80,467793.0,...,B,2.39,7.56,3.38,1.63,631549.0,2.505,"(2.39, 7.56)","(0.0, 2.505)",5.591523
274,2020-09-27,Duane Underwood Jr.,SL,87.2,ball,3.0,14.0,-1.82,6.24,673357.0,...,B,3.69,-1.50,3.59,1.76,631488.0,2.675,"(3.69, -1.5)","(0.0, 2.675)",5.571959
16284,2020-09-10,Robbie Erlin,CU,75.7,blocked_ball,2.0,13.0,2.52,5.68,594809.0,...,B,-2.92,-2.41,2.98,1.40,630869.0,2.190,"(-2.92, -2.41)","(0.0, 2.19)",5.448523
5004,2020-09-05,Anthony Bass,SL,86.6,blocked_ball,3.0,14.0,-0.58,5.63,622168.0,...,B,2.52,-2.17,3.18,1.55,631625.0,2.365,"(2.52, -2.17)","(0.0, 2.365)",5.188123
15822,2020-09-17,Griffin Canning,CU,87.6,blocked_ball,4.0,14.0,-1.85,5.90,500871.0,...,B,0.93,-2.72,3.12,1.53,631362.0,2.325,"(0.93, -2.72)","(0.0, 2.325)",5.130002


In [14]:
df.description.value_counts()

ball                       16567
foul                       10767
called_strike               7130
hit_into_play               7032
swinging_strike             6401
blocked_ball                1930
hit_into_play_no_out        1613
hit_into_play_score         1390
swinging_strike_blocked      577
foul_tip                     569
hit_by_pitch                 159
foul_bunt                     36
pitchout                       6
missed_bunt                    3
Name: description, dtype: int64

In [50]:
df[(df.description.isin(['hit_into_play',
                        'hit_into_play_no_out',
                        'hit_into_play_score',
                        'foul_tip',
                        'swinging_strike_blocked',
                        'swinging_strike'])) 
].sort_values("dist_center", ascending=False).sort_values("dist_center", ascending=False)

,game_date,player_name,pitch_type,release_speed,description,pitch_number,zone,release_pos_x,release_pos_z,batter,...,type,plate_x,plate_z,sz_top,sz_bot,game_pk,sz_center,pitch_plate,center_plate,dist_center
19358,2020-09-23,Tyler Glasnow,CU,84.5,swinging_strike,3.0,14.0,-1.49,6.18,624413.0,...,S,1.74,-1.58,3.44,1.66,631218.0,2.550,"(1.74, -1.58)","(0.0, 2.55)",4.481573
17830,2020-09-23,Trevor Bauer,KC,80.8,swinging_strike_blocked,3.0,14.0,-1.25,5.89,592885.0,...,S,1.08,-1.33,3.28,1.72,631583.0,2.500,"(1.08, -1.33)","(0.0, 2.5)",3.979359
4966,2020-09-29,Tyler Duffey,KC,83.6,swinging_strike_blocked,7.0,14.0,-2.88,5.70,621043.0,...,S,2.41,-0.39,3.47,1.67,635907.0,2.570,"(2.41, -0.39)","(0.0, 2.5700000000000003)",3.817028
13560,2020-07-29,Lucas Sims,CU,81.2,swinging_strike_blocked,4.0,14.0,-1.12,5.42,664023.0,...,S,1.79,-0.79,3.37,1.53,631594.0,2.450,"(1.79, -0.79)","(0.0, 2.45)",3.701581
16706,2020-09-09,Jason Adam,CU,82.4,swinging_strike,2.0,14.0,-1.90,5.90,608385.0,...,S,1.68,-0.78,3.42,1.61,631616.0,2.515,"(1.68, -0.78)","(0.0, 2.515)",3.698571
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
16740,2020-09-17,Chasen Shreve,FF,92.0,foul_tip,6.0,5.0,2.03,5.75,457705.0,...,S,0.02,2.48,3.38,1.61,631117.0,2.495,"(0.02, 2.48)","(0.0, 2.495)",0.025000
4392,2020-08-08,Josh Osich,FF,92.7,hit_into_play,2.0,5.0,1.69,6.15,624415.0,...,X,-0.01,2.47,3.40,1.58,631658.0,2.490,"(-0.01, 2.47)","(0.0, 2.49)",0.022361
10250,2020-08-28,Emilio Pagan,FC,85.9,hit_into_play,5.0,5.0,-1.73,5.47,453568.0,...,X,-0.02,2.59,3.53,1.66,631524.0,2.595,"(-0.02, 2.59)","(0.0, 2.5949999999999998)",0.020616
3219,2020-08-16,Tyler Heineman,CU,69.6,hit_into_play,3.0,5.0,-1.18,6.50,620439.0,...,X,-0.02,2.40,3.30,1.50,631024.0,2.400,"(-0.02, 2.4)","(0.0, 2.4)",0.020000
